# FAISS 벡터스토어
이 노트북에서는 LangChain과 FAISS를 사용하여 고성능 벡터스토어를 생성하고 활용하는 방법을 학습합니다.
### FAISS란?
`FAISS(Facebook AI Similarity Search)`는 Facebook AI Research에서 개발한 효율적인 유사도 검색 및 군집화 라이브러리입니다.



In [ ]:
# FAISS 설치
%pip install faiss-cpu

In [ ]:
# 필요한 라이브러리 및 데이터 준비
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 훈련 로그 데이터 로드
loader = TextLoader("data/training_log_all.txt")
docs = loader.load()

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

print(f"총 {len(split_docs)}개의 문서 청크 준비 완료")
print(f"첫 번째 문서: {split_docs[0].page_content[:100]}...")


In [ ]:
# 임베딩 모델 설정
from langchain_huggingface import HuggingFaceEmbeddings
# 한국어 특화 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",  # 한국어 특화 임베딩 모델
    model_kwargs={'device': 'cpu'},  # CPU 사용 (GPU가 있다면 'cuda'도 가능)
    encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화 활성화
)

print("한국어 임베딩 모델 설정 완료")


### 1. 기본 FAISS 벡터스토어 생성
대규모 데이터셋에서 빠른 유사도 검색이 가능합니다.


In [ ]:
# FAISS 벡터스토어 생성
from langchain_community.vectorstores import FAISS

# 문서에서 FAISS 벡터스토어 생성
db = FAISS.from_documents(
    documents=split_docs,
    embedding=embedding_model
)

print("FAISS 벡터스토어 생성 완료!")
print(f"저장된 문서 수: {len(split_docs)}")

# 벡터스토어 정보 확인
print(f"임베딩 차원: {db.index.d}")
print(f"인덱스 크기: {db.index.ntotal}")


### 2. 유사도 검색 (Similarity Search)

FAISS는 매우 빠른 유사도 검색을 제공합니다. 다양한 검색 방법을 실습해보겠습니다.

- **similarity_search**: 기본 유사도 검색
- **similarity_search_with_score**: 점수와 함께 검색
- **max_marginal_relevance_search**: 다양성을 고려한 검색


In [ ]:
# 기본 유사도 검색
query = "2대대의 분대 전술 훈련이 언제 진행되었나요?"

# 가장 유사한 3개 문서 검색
search_results = db.similarity_search(
    query=query,
    k=3  # 상위 3개 결과 반환
)

print(f"검색 질의: '{query}'")
print("검색 결과:")
for i, doc in enumerate(search_results):
    print(f"\n{i+1}. 내용: {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")


In [ ]:
# 유사도 점수와 함께 검색
query2 = "23일에는 어떤 훈련이 진행되었나요?"

# 유사도 점수와 함께 검색 (점수가 낮을수록 더 유사함)
search_results_with_score = db.similarity_search_with_score(
    query=query2,
    k=3
)

print(f"검색 질의: '{query2}'")
print("점수별 검색 결과:")
for i, (doc, score) in enumerate(search_results_with_score):
    print(f"\n{i+1}. 유사도 점수: {score:.4f}")
    print(f"   내용: {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")


### 3. 새로운 문서 추가
FAISS 벡터스토어에 새로운 문서를 동적으로 추가할 수 있습니다.

In [ ]:
# 새로운 문서 추가
from langchain_core.documents import Document

# 새로운 문서 생성
new_documents = [
    Document(
        page_content="7월 19일 금요일 식단은 아침에 북어국과 계란말이, 점심에는 치킨마요 덮밥과 단무지, 저녁에는 부대찌개와 김치전이 제공됩니다.",
        metadata={"category": "food", "date": "2025-07-19", "day": "Friday"}
    ),
    Document(
        page_content="7월 20일 토요일 식단은 아침에 미역국과 스크램블에그, 점심에는 제육볶음과 콩나물국, 저녁에는 김치찌개와 어묵볶음이 나옵니다.",
        metadata={"category": "food", "date": "2025-07-20", "day": "Saturday"}
    ),
    Document(
        page_content="7월 21일 일요일 식단은 아침에 떡국과 김, 점심에는 함박스테이크와 감자조림, 저녁에는 라면과 김밥이 제공됩니다.",
        metadata={"category": "food", "date": "2025-07-21", "day": "Sunday"}
    )
]

# 기존 벡터스토어에 새 문서 추가
db.add_documents(new_documents)

print("새로운 문서 3개 추가 완료!")
print(f"현재 총 문서 수: {db.index.ntotal}")

### 4. MMR 방식 서칭
`max_marginal_relevance_search` 
- 메서드는 검색 결과의 다양성을 고려하여 문서를 반환하는 검색 방식입니다.

주요 파라미터:
- query: 검색할 쿼리 문자열
- k: 최종적으로 반환할 문서의 수 
- fetch_k: 초기에 검색할 후보 문서의 수 (k보다 커야 함)
- lambda_mult: 유사도와 다양성 사이의 가중치 (0~1 사이 값)
  - 1에 가까울수록 쿼리와의 유사도를 중시
  - 0에 가까울수록 검색 결과의 다양성을 중시

작동 방식:
1. fetch_k개의 후보 문서를 쿼리와의 유사도 기준으로 검색
2. 이미 선택된 문서들과의 유사도를 고려하여 다양성이 높은 문서를 k개 선택
3. lambda_mult를 통해 '쿼리와의 유사도'와 '기존 선택 문서들과의 차이'의 중요도를 조절  

| 값 (`lambda_mult`) | 의미                        | 설명 |
|--------------------|-----------------------------|------|
| `1.0`              | 최대 유사성 (relevance)     | 쿼리에 가장 유사한 문서만 고름 |
| `0.0`              | 최대 다양성 (diversity)     | 서로 다른 내용의 문서를 골라 결과를 다양하게 함 |
| `0.3 ~ 0.7`        | 유사성과 다양성의 절충      | 일반적으로 가장 유용한 범위 |


In [ ]:
# MMR(Maximal Marginal Relevance) 검색 - 다양성을 고려한 검색
query3 = "20"

# MMR 검색 실행
mmr_results = db.max_marginal_relevance_search(
    query=query3,
    k=2,  # 반환할 문서 수
    fetch_k=5,  # 후보 문서 수 (더 많은 후보에서 다양성 고려)
    lambda_mult=0  # 다양성 조절 (0: 최대 다양성, 1: 최대 유사성)
)

print(f"MMR 검색 질의: '{query3}'")
print("\n다양성을 고려한 검색 결과:")
for i, doc in enumerate(mmr_results):
    print(f"\n{i+1}. 내용: {doc.page_content[:100]}...")
    print(f"   메타데이터: {doc.metadata}")

print(f"MMR은 유사도와 다양성을 모두 고려하여 중복되지 않는 다양한 관점의 문서를 반환합니다.")


### 5. 로컬 저장 및 로드
FAISS 벡터스토어를 로컬 파일 시스템에 저장하고 불러올 수 있습니다. 이를 통해 다음 세션에서도 동일한 데이터를 사용할 수 있습니다.

In [ ]:
# 로컬 저장
save_path = "./faiss_db"

# FAISS 인덱스를 로컬에 저장
db.save_local(save_path)


In [ ]:
# 저장된 벡터스토어 로드
loaded_db = FAISS.load_local(
    save_path, 
    embedding_model,
    allow_dangerous_deserialization=True  # 로컬 파일 역직렬화 허용
)

# 로드된 벡터스토어로 검색 테스트
load_test_query = "식단"
load_test_results = loaded_db.similarity_search(load_test_query, k=2)

print(f"로드된 DB 검색 테스트: '{load_test_query}'")
for i, doc in enumerate(load_test_results):
    print(f"{i+1}. {doc.page_content[:50]}...")
    if 'category' in doc.metadata:
        print(f"   카테고리: {doc.metadata['category']}")